## R - Maps and Overlaying Spatial Data


In this notebook, we will see examples of how to obtain map images and visualize data as layers on them using ggplot and Google maps. 

Take a look at the [ggmap reference](https://journal.r-project.org/archive/2013-1/kahle-wickham.pdf).
 * [Local Mirror](/static/PDF/ggmap_kahle_wickham.pdf)

In [ ]:
library(ggplot2)
library(ggmap)
library(maps)
library(maptools)
library(RgoogleMaps)
library(sp)

There are different ways of getting map images and displaying them in R. 
We will start with **qmap()**; it is an easy mapping tool using ggmap library. 

**Note:** When utilizing map services, such as GoogleMaps and others, often the "query" includes a location element and/or a `zoom` level.
The zoom level dictates how close or far the visual dipiction is for the map rendering.

# <span style="background:yellow">Your Turn</span>

Adjust the zoom level and zip code to get a feel for different zoom levels.
You may want to switch to a zip code you are more familiar with, such as your current home or work.

In [ ]:
# Alter the zip code and the zoom parameter 
# a few times to get a feel for the changes.
qmap('65202', zoom = 10)

In [ ]:
qmap('65201', zoom = 14, maptype = 'satellite')

In [ ]:
qmap('65201', zoom = 14, maptype = 'hybrid')

**get_map()** is a general function to grab map images from many online sources including Google Maps. If we want to specifically use Google Maps, then we will use **GetMap()** function of RgoogleMaps library. 

In [ ]:
# Let's get a map of Houston
hdf <- get_map("houston, texas")
# and display it 
ggmap(hdf, extent = "device")

## Overlaying addition data

The below examples demonstrate stacking layers of information on top of the map.
In the cases below, you will see similar techniques and concepts as previously shown in ggplot2; 
however, now the positional aspects of the data are geospatial instead of other data attributes.

In [ ]:
# Let's make some fake spatial data to show how to display it on the map 
mu <- c(-95.3632715, 29.7632836); nDataSets <- sample(4:10,1)
chkpts <- NULL
for(k in 1:nDataSets){
 a <- rnorm(2); b <- rnorm(2);
 si <- 1/3000 * (outer(a,a) + outer(b,b))
 chkpts <- rbind(chkpts,cbind(MASS::mvrnorm(rpois(1,50), jitter(mu, .01), si), k))
}

chkpts <- data.frame(chkpts)
names(chkpts) <- c("lon", "lat","class")
chkpts$class <- factor(chkpts$class)

# qmplot is easier to use in this case: it'l collect the map from an online resource
qmplot(lon, lat, data = chkpts, color = class, darken = .6)

In [ ]:

# Let's create a density plot:
qmplot(lon, lat, data = chkpts, geom = "density2d", color = class, darken = .6)

In [ ]:
# or overlay it on a satellite image with ggplot 
ggmap(get_map(maptype = "satellite"), extent = "device") +
stat_density2d(aes(x = lon, y = lat, colour = class), data = chkpts, bins = 5)

### We can also create a traditional geospatial vector layers on top of satellite images.

We will brind into the code the built-in **zips** data set.  
**Note**: It is specifically just Houston, Texas zip code data.

In [ ]:
data(zips)
help(zips)
str(zips)

In [ ]:

ggmap(get_map(maptype = "satellite", zoom = 8), extent = "device") +
geom_polygon(aes(x = lon, y = lat, group = plotOrder), data = zips, colour = NA, fill = "red", alpha = .2) +
geom_path(aes(x = lon, y = lat, group = plotOrder), data = zips, colour = "white", alpha = .4, size = .4)

## Visual Analytical Maps

The subsequent maps show various techniques to render geospatial data over maps using crime data from the downtown Houston area.

After rendering each cell, please carefully review the code. 
Look for differences and similarities in what the various aspects of the code and the associated output.

In [ ]:
## Let's do a crime plot on downtown Houston 

# pick only violent crimes
violent_crimes <- subset(crime,offense != "auto theft" & offense != "theft" & offense != "burglary")
# rank violent crimes
violent_crimes$offense <- factor(violent_crimes$offense, levels = c("robbery", "aggravated assault","rape", "murder"))

# restrict to downtown
violent_crimes <- subset(violent_crimes, -95.39681 <= lon & lon <= -95.34188 & 29.73631 <= lat & lat <= 29.78400)

# get map and bounding box
theme_set(theme_bw(16))

HoustonMap <- ggmap(get_map("houston", zoom = 14, color = "bw"),extent = "device", legend = "topleft")

# the bubble chart
HoustonMap +
geom_point(aes(x = lon, y = lat, colour = offense, size = offense), data = violent_crimes, alpha=0.6) +
scale_colour_discrete("Offense", labels = c("Robbery","Aggravated Assault","Rape","Murder")) +
scale_size_discrete("Offense", labels = c("Robbery","Aggravated Assault","Rape","Murder"), range = c(1.75,6)) +
guides(size = guide_legend(override.aes = list(size = 6))) +
theme(
   legend.key.size = grid::unit(1.8,"lines"),
   legend.title = element_text(size = 16, face = "bold"),
   legend.text = element_text(size = 14)
   ) +
labs(colour = "Offense", size = "Offense")

In [ ]:
# doing it with qmplot is even easier - let's use a toner friendly map 
qmplot(lon, lat, data = violent_crimes, maptype = "toner-lite", color = offense, alpha = 0.8, size = offense, legend = "topleft")


## Contour plot overlay

In [ ]:
# Let's do a contour plot
HoustonMap +
stat_density2d(aes(x = lon, y = lat, colour = offense),
size = 3, bins = 2, alpha = 3/4, data = violent_crimes) +
scale_colour_discrete("Offense", labels = c("Robery","Aggravated Assault","Rape","Murder")) +
theme(
   legend.text = element_text(size = 15, vjust = .5),
   legend.title = element_text(size = 15,face="bold"),
   legend.key.size = grid::unit(1.8,"lines")
)

## 2-D histogram
Effectively, a discrete bucket heatmap.

In [ ]:
# and a 2d histogram...
HoustonMap +
stat_bin2d(aes(x = lon, y = lat, colour = offense, fill = offense), size = .5, bins = 30, alpha = 0.4, data = violent_crimes) +
scale_colour_discrete("Offense",labels = c("Robery","Aggravated Assault","Rape","Murder"),guide = FALSE) +
scale_fill_discrete("Offense", labels = c("Robery","Aggravated Assault","Rape","Murder")) +
theme(
   legend.text = element_text(size = 15, vjust = .5),
   legend.title = element_text(size = 15,face="bold"),
   legend.key.size = grid::unit(1.8,"lines")
)

You may notice all the messages produced by different map servers get_map() is getting data from. To suppress them, you can use the suppressMessages() function. Let's do a crime density visualization.

## Density Regions

In [ ]:
# Let's get a color map and do a density plot with stat_density2d
houston <- suppressMessages(get_map("houston", zoom = 14))
HoustonMap <- ggmap(houston, extent = "device", legend = "topleft", darken = c(.5,"white")) # this whitens the map 

HoustonMap +
stat_density2d(aes(x = lon, y = lat, fill = ..level.., alpha = ..level..),size = 2, bins = 4, data = violent_crimes, geom = "polygon") +
scale_fill_gradient("Violent\nCrime\nDensity",low = "green", high = "red") + 
scale_alpha(range = c(0.4, 0.7), guide = FALSE) +
guides(fill = guide_colorbar(barwidth = 1.5, barheight = 10))


In [ ]:
# Let's do a finer plot with 16 bins and add boundaries too
HoustonMap +
geom_density2d(data = violent_crimes, aes(x = lon, y = lat), size = 0.3) + 
stat_density2d(data = violent_crimes, aes(x = lon, y = lat, fill = ..level.., alpha = ..level..), size = 0.01, bins = 16, geom = "polygon") + 
scale_fill_gradient("Violent\nCrime\nDensity",low = "green", high = "red") + 
scale_alpha(range = c(0.1, 0.4), guide = FALSE)

# RgoogleMaps
RgoogleMaps library also has its own functions to visualize maps and data. 

In [ ]:
# This is how to obtain a map image using RgoogleMaps library 

# Jesse Hall coordinates
lat = c(38.945092);
lon = c(-92.328806);
center = c(mean(lat), mean(lon));

Map <- GetMap(center=center, zoom=14);

PlotOnStaticMap(Map, lat = lat, lon = lon, cex=1.5,pch=20, col=c('red'), add=FALSE);


In [ ]:
# This is an example of how to find the center and zoom from the geo data, 
# and how to add Google-style markers to the map 
lat = c(40.702147,40.711614,40.718217);
lon = c(-74.015794,-74.012318,-73.998284);
center = c(mean(lat), mean(lon));
zoom <- min(MaxZoom(range(lat), range(lon)));

MyMap <- GetMap(center=center, zoom=zoom,markers = paste0("&markers=color:blue|label:S|",
           "40.702147,-74.015794&markers=color:green|label:G|40.711614,-74.012318&markers=",
           "color:red|color:red|label:C|40.718217,-73.998284"), destfile = "MyTile1.png");

PlotOnStaticMap(MyMap, lat = lat, 
                       lon = lon, 
                       destfile = "MyTile1.png", cex=1.5,pch=20,
                       col=c('red', 'blue', 'green'), add=FALSE);

#and add lines:
PlotOnStaticMap(MyMap, lat = c(40.702147,40.711614,40.718217), 
                       lon = c(-74.015794,-74.012318,-73.998284), 
                       lwd=1.5,col=c('red', 'blue', 'green'), FUN = lines, add=TRUE)


In [ ]:
# This data set comes with the RgoogleMaps library 
data("NYleukemia")

# Let's visualize the locations of cases using RgoogleMaps functions
head(NYleukemia$data)
population <- NYleukemia$data$population
cases <- NYleukemia$data$cases

geo <- NYleukemia$geo
head(geo)
lats <-geo$y
lons <-geo$x

mapNY <- GetMap(center=c(lat=42.67456,lon=-76.00365), destfile = "NYstate.png", maptype = "street", zoom=9)
PlotOnStaticMap(mapNY, lat = lats, lon = lons, cex=2, pch=20, col="red", add=FALSE);

In [ ]:
# Let's do it properly in ggplot

NY <- suppressMessages(get_map("homer, new york", zoom = 9))
NYmap <- ggmap(NY, extent = "device", legend = "topleft", darken = c(.5,"white"))

NYdf <- data.frame(population, cases, lats, lons)
head(NYdf)

In [ ]:
NYmap + 
geom_point(data=NYdf, aes(x = lons, y = lats, color = cases, size = cases), alpha=0.7) + 
scale_color_gradient(low = "yellow", high = "red") + 
scale_alpha(range = c(0.1, 0.4), guide = FALSE)

# Save your notebook, then File > Close and Halt